In [20]:
%reset
from sklearn.cluster import k_means
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from kmodes.kprototypes import KPrototypes
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import numpy as np
from plotnine import *
import plotnine
from matplotlib import pyplot as plt
import pickle

In [21]:
df = pd.read_csv("stanbic.csv")
# enc = OneHotEncoder(sparse=False)
# df['profession_subclass'] = df['profession_subclass'].str.replace(' ',"_")
# print(df['profession_subclass'])
df = df.iloc[:,:-3]
df.credit_cards = df.credit_cards.apply(lambda x:x.strip("[]"))
# print(df.credit_cards.shape)
df_test = df.credit_cards.str.get_dummies(sep = ',')
# print(df.dtypes)
df = df.drop(['credit_cards'],axis=1)
df = df.join(df_test)
# df['profession_subclass'] = enc.fit_transform(np.array(df['profession_subclass']).reshape(-1,1))
df = df.drop('cust_id',axis=1)
# df['credit_cards'] = enc.fit_transform(np.array(df['credit_cards']).reshape(-1,1))

<h1>Regressor using Light GBM</h1>

In [22]:
obj_feat = list(df.loc[:, df.dtypes == 'object'].columns.values)
for feature in obj_feat:
    df[feature] = pd.Series(df[feature], dtype="category")

In [23]:
df.amt_spent_cat3.dtypes

dtype('int64')

In [24]:
x_train,x_test,y_train,y_test = train_test_split(df.iloc[:,:44],df.iloc[:,80],test_size=0.20,random_state=33)
model = lgb.LGBMRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pr = y_pred
y_pr[y_pr>=0.5]=1
y_pr[y_pr<0.5]=0
print(accuracy_score(y_test,y_pr))
fea_imp_ = pd.DataFrame({'cols':x_test.columns, 'fea_imp':model.feature_importances_})
fea_imp_.loc[fea_imp_.fea_imp > 100].sort_values(by=['fea_imp'], ascending = False)

0.9885167464114832


,cols,fea_imp
36,amt_spent_cat3,264
34,amt_spent_cat1,260
41,max_amt_in_curr_acc,244
39,max_amt_in_sav_acc,236
35,amt_spent_cat2,219
38,amt_in_sav_acc,213
20,income,198
40,amt_in_curr_acc,155
21,credit_score,147
37,total_no_of_txn,108


In [25]:
list = fea_imp_.loc[fea_imp_.fea_imp > 100].sort_values(by=['fea_imp'], ascending = False).cols
list = np.array(list)
list[1]

'amt_spent_cat1'

In [26]:
np.array(df.iloc[1,:45]).reshape(1,-1)

array([['Female', '30-39', 34, 'Ugandan', 'Silver', 10, 'Mbarara',
        'Mbarara', 0, 0, 'Single', 1, 'Lawyer',
        'Internship Legal Officer', 0, 0, 0, 0, 1, 0, 2850549, 629, 1, 2,
        12, 3, 2, 2, 'Bills', 'Dining', 'Golf', 18, 9, 9, 90775, 62595,
        58496, 22, 188105, 317929, 884122, 901152, 7, 20, 1]],
      dtype=object)

In [27]:
acc_score = []
name = []

for i in range(46,90):
    x1_train,x1_test,y1_train,y1_test = train_test_split(df.iloc[:,:45],df.iloc[:,i],test_size=0.20,random_state=33)
    model = lgb.LGBMRegressor()
    md_name = df.iloc[:,i].name
    name.append(md_name)
    filename = f'/home/abhinav-dev/reco-task/Models_cards/Model_{md_name}.sav'
    model.fit(x1_train, y1_train)
    pickle.dump(model, open(filename, 'wb'))
    y_pred = model.predict(x1_test)
    y_pr = y_pred
    y_pr[y_pr>=0.5]=1
    y_pr[y_pr<0.5]=0
    acc_score.append(accuracy_score(y1_test,y_pr)*100)
    print(accuracy_score(y1_test,y_pr))

d = {'name':name,'accuracy':acc_score}
df_acc = pd.DataFrame(data=d)
df_acc.to_excel("Model_Performance.xlsx")

0.9263157894736842
0.7684210526315789
0.8832535885167464
0.9626794258373206
0.9454545454545454
0.9837320574162679
0.9483253588516747
0.9722488038277513
0.9846889952153111
0.9607655502392345
0.7004784688995215
0.876555023923445
0.9052631578947369
0.724401913875598
0.999043062200957
0.7674641148325358
0.9177033492822967
0.9531100478468899
0.9148325358851674
0.9301435406698565
0.9636363636363636
0.9186602870813397
0.999043062200957
0.9980861244019139
0.9540669856459331
0.6555023923444976
0.9980861244019139
0.9894736842105263
0.9904306220095693
1.0
0.9760765550239234
0.9980861244019139
0.999043062200957
0.9913875598086125
0.9885167464114832
0.9330143540669856
0.6784688995215311
0.9531100478468899
0.9425837320574163
0.9521531100478469
0.9923444976076555
0.9205741626794258
1.0
0.9942583732057416


In [28]:
sum(acc_score)/len(acc_score)

92.75337103088302

In [29]:
op = df.iloc[:1,:45]
type(op)

pandas.core.frame.DataFrame

In [30]:
reco = []
for i in range(46,90):
    md_name = df.iloc[:,i].name
    filename = f'/home/abhinav-dev/reco-task/Models_cards/Model_{md_name}.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred = loaded_model.predict(df.iloc[[2],:45])
    if(y_pred >= 0.5):
        reco.append(md_name)

In [31]:
reco

[" 'Corporate Premium Credit Card'",
 " 'Diners Club Premium Credit Card'",
 " 'Visa Signature Credit Card'",
 "'Infinite Credit Card'"]

<h1>Kprototypes for clustering(not effective)</h1>

In [32]:
%%script echo skipping
catColumnsPos = [df.columns.get_loc(col) for col in list(df.select_dtypes('object').columns)]

skipping


In [33]:
%%script echo skipping
cost = []
dfMatrix = df.to_numpy()
for cluster in range(1, 10):
    kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
    kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)
    cost.append(kprototype.cost_)
    print('Cluster initiation: {}'.format(cluster))

skipping


In [34]:
%%script echo skipping
plt.plot(cost)
plt.show()

skipping


In [35]:
%%script echo skipping
kprototype = KPrototypes(n_jobs = -1, n_clusters = 2, init = 'Huang', random_state = 0)
kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)

skipping


In [36]:
%%script echo skipping
kprototype.predict(df.iloc[:1000],categorical = catColumnsPos)

skipping
